In [1]:
import sys
import numpy as np
import cv2

In [2]:
oldx, oldy = -1, -1

# 마우스에 대한 콜백처리 함수
def on_mouse(event, x, y, flags, _):
   global oldx, oldy

   if event == cv2.EVENT_LBUTTONDOWN:
      oldx, oldy = x, y

   elif event == cv2.EVENT_LBUTTONUP:
      oldx, oldy = -1, -1

   elif event == cv2.EVENT_MOUSEMOVE:
      if flags & cv2.EVENT_FLAG_LBUTTON:
         cv2.line(img, (oldx, oldy), (x, y), (255, 255, 255), 40, cv2.LINE_AA)
         oldx, oldy = x, y
         cv2.imshow('img', img)

In [3]:
# 학습 & label 행렬 생성

digits = cv2.imread('./digits.png', cv2.IMREAD_GRAYSCALE)

if digits is None:
    print('Image load failed!')
    sys.exit()

h, w = digits.shape[:2]
# 여기서 h = 1000(20x50), w = 2000(20x100)

# 각각의 부분 영상을 잘라내는 코드
cells = [np.hsplit(row, w//20) for row in np.vsplit(digits, h//20)]
# h/20 = 50 (세로 50개의 숫자)   w/20 = 100 (가로 100개의 숫자)
# 여기서 cells 는 list 형태로 저장된다

cells = np.array(cells)
# list의 형태였던 것을 array 형태로 변환

train_images = cells.reshape(-1, 400).astype(np.float32)
# 20X20짜리 행렬을 400X1 행렬로 reshape

train_labels = np.repeat(np.arange(10), len(train_images)/10)
# 정답에 해당하는 label을 지정해 주는 코드

In [4]:
# KNN 학습

knn = cv2.ml.KNearest_create()
knn.train(train_images, cv2.ml.ROW_SAMPLE, train_labels)

True

In [5]:
# 사용자 입력 영상에 대해 예측

img = np.zeros((400, 400), np.uint8)
# 400X400 비어있는 영상

cv2.imshow('img', img)
cv2.setMouseCallback('img', on_mouse)
# 마우스 콜백 함수 등록

while True:
    key = cv2.waitKey()

    if key == 27:
        break
    elif key == ord(' '):
        test_image = cv2.resize(img, (20, 20), interpolation=cv2.INTER_AREA)
        test_image = test_image.reshape(-1, 400).astype(np.float32)

        ret, _, _, _ = knn.findNearest(test_image, 5)
        # test_image의 값을 knn알고리즘에 넣고 가장 가까운 5개(k=5)의 글자를 찾아, 어떤 숫자에 해당하는지 찾는다 
        # test로 준 test_image 샘플은 하나이기 때문에 res 두 번째 행렬로 안받고 return값 ret로 받아서 
        print(int(ret))
        # 정수로 출력

        img.fill(0)
        # 출력 후에는 다시 화면을 까맣게 채워준다
        cv2.imshow('img', img)

cv2.destroyAllWindows()

1
1
2
3
